In [1]:
import json
import yaml
import requests

with open('../keys/ip_addresses.yaml', 'r') as f:
    hundley_mac_mini_ip = yaml.safe_load(f)['HUNDLEY_MAC_MINI_LOCAL_IP']

# API endpoint
url = f"http://{hundley_mac_mini_ip}:8585/v1/chat/completions"

# Headers
headers = {
    "Content-Type": "application/json"
}

# Request payload
payload = {
    "messages": [{"role": "user", "content": "Say this is a test!"}],
    "temperature": 0.7
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(payload))

# Print the response
print(response.status_code)
print(response.json())


200
{'id': 'chatcmpl-f93cc5d5-9cbf-4142-8330-fd5f174a6517', 'system_fingerprint': '0.21.5-0.23.2-macOS-15.3.1-arm64-arm-64bit-applegpu_g16g', 'object': 'chat.completion', 'model': 'default_model', 'created': 1741972254, 'choices': [{'index': 0, 'logprobs': {'token_logprobs': [-0.234375, 0.0, -0.328125, -0.859375, -0.09375, -0.75, 0.0, 0.0, -0.0625, -0.421875, -0.25, 0.0, -0.015625, 0.0, -0.65625, -1.84375, -0.09375, -0.703125, -0.21875, -0.0625, -0.53125, -2.3125, -0.265625, -0.25, 0.0, -4.1328125, 0.0, 0.0, 0.0, -0.25, -0.0625], 'top_logprobs': [], 'tokens': [7373, 8999, 29576, 1619, 1117, 8934, 1032, 2137, 29491, 2370, 1309, 1083, 6799, 1136, 1163, 1342, 2137, 1210, 2084, 1136, 3148, 1032, 6703, 2641, 29572, 3962, 2114, 1296, 1641, 29576, 2]}, 'finish_reason': 'stop', 'message': {'role': 'assistant', 'content': 'Understood! This is indeed a test. How can I assist you with your test or help you understand a concept better? Just let me know!'}}], 'usage': {'prompt_tokens': 11, 'complet

In [2]:
import requests
import json
import litellm
from litellm import CustomLLM
import dspy
import uuid
from time import time

class MyCustomAPILLM(CustomLLM):
    def __init__(self):
        super().__init__()
        self.url = f"http://{hundley_mac_mini_ip}:8585/v1/chat/completions"
        self.headers = {
            "Content-Type": "application/json"
        }
        
    def _process_messages(self, messages):
        """Process messages to ensure they're in the correct format"""
        processed_messages = []
        
        # Extract any system messages
        system_content = ""
        for msg in messages:
            # Convert message objects to dicts if needed
            if not isinstance(msg, dict):
                if hasattr(msg, "role") and hasattr(msg, "content"):
                    msg = {"role": msg.role, "content": msg.content}
                else:
                    continue
            
            # Handle system messages
            if msg.get("role") == "system":
                system_content += msg.get("content", "") + "\n"
                continue
                
            # Keep user and assistant messages
            if msg.get("role") in ["user", "assistant"]:
                processed_messages.append({
                    "role": msg["role"],
                    "content": msg["content"]
                })
        
        # Ensure we start with a user message
        if not processed_messages or processed_messages[0]["role"] != "user":
            processed_messages.insert(0, {"role": "user", "content": "Hello"})
        
        # Add system content to first user message if exists
        if system_content and processed_messages:
            for i, msg in enumerate(processed_messages):
                if msg["role"] == "user":
                    processed_messages[i]["content"] = system_content.strip() + "\n\n" + msg["content"]
                    break
        
        # Ensure alternating pattern
        filtered_messages = []
        expected_role = "user"
        for msg in processed_messages:
            if msg["role"] == expected_role:
                filtered_messages.append(msg)
                expected_role = "assistant" if expected_role == "user" else "user"
        
        return filtered_messages
        
    def _format_openai_response(self, raw_response):
        """Convert the API response to OpenAI format"""
        try:
            # Create a properly formatted response
            formatted_response = {
                "id": str(uuid.uuid4()),  # Generate a unique ID string
                "object": "chat.completion",
                "created": int(time()),
                "model": "custom-model",
                "choices": [
                    {
                        "index": 0,
                        "message": {
                            "role": "assistant",
                            "content": raw_response.get("choices", [{}])[0].get("message", {}).get("content", "")
                        },
                        "finish_reason": "stop"
                    }
                ],
                "usage": {
                    "prompt_tokens": 0,
                    "completion_tokens": 0,
                    "total_tokens": 0
                }
            }
            
            # Try to get usage from the response if available
            if "usage" in raw_response:
                formatted_response["usage"] = raw_response["usage"]
                
            return formatted_response
        except Exception as e:
            raise Exception(f"Error formatting response: {str(e)}")
            
    def completion(self, model, messages, temperature=0.7, max_tokens=None, **kwargs):
        """Sends completion request to custom API endpoint"""
        try:
            # Only include keys that are expected by your API
            ignored_keys = ["model_response", "print_verbose", "acompletion", "logging_obj", 
                           "optional_params", "litellm_params", "logger_fn", "custom_prompt_dict", 
                           "client", "encoding"]
            
            # Process messages to ensure proper format
            processed_messages = self._process_messages(messages)
            
            # Build clean payload
            payload = {
                "messages": processed_messages,
                "temperature": temperature
            }
            
            # Add max_tokens if provided
            if max_tokens is not None:
                payload["max_tokens"] = max_tokens
            
            # Add basic parameters that your API might support
            for key, value in kwargs.items():
                if key not in ignored_keys and not callable(value):
                    payload[key] = value
            
            # Make the API call
            response = requests.post(
                self.url, 
                headers=self.headers, 
                json=payload  # Using json parameter handles serialization for you
            )
            
            # Check if the request was successful
            if response.status_code == 200:
                # Get raw response data
                raw_response = response.json()
                
                # Format the response to match OpenAI's structure
                formatted_response = self._format_openai_response(raw_response)
                
                # Create a ModelResponse using the formatted response
                return litellm.ModelResponse(**formatted_response)
            else:
                # Handle API errors
                raise Exception(f"API request failed with status code: {response.status_code}, details: {response.text}")
                
        except Exception as e:
            # Provide detailed error information
            raise Exception(f"Error in custom LLM handler: {str(e)}")

# Create an instance of your custom LLM
my_custom_llm = MyCustomAPILLM()

# Register your custom provider with LiteLLM
litellm.custom_provider_map = [
    {"provider": "my-custom-api", "custom_handler": my_custom_llm}
]

# Set up DSPy to use your custom LLM
llm = dspy.LM(
    model="my-custom-api/custom-model",
    temperature=0.7,
    max_tokens=1000
)

dspy.configure(lm = llm)

# Creating sample sentences representing positive and negative sentiment
positive_sentence = "I am very happy with the results of this project."
negative_sentence = "I am disappointed with the outcome of this task."

# Instantiating a simple DSPy module for sentiment classification
dspy_sentiment_classification = dspy.Predict('sentence -> sentiment: bool')

# Invoking the DSPy model with each respective sentence.
print(f'Positive sentence: {dspy_sentiment_classification(sentence = positive_sentence)}')
print(f'Negative sentence: {dspy_sentiment_classification(sentence = negative_sentence)}')

/Users/dkhundley/Documents/Repositories/dspy-tutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Positive sentence: Prediction(
    sentiment=True
)
Negative sentence: Prediction(
    sentiment=True
)


In [4]:
# Creating a class-based DSPy signature for text analysis
class TextAnalysisSignature(dspy.Signature):
    """Analyze text for sentiment, main topic, and formality level."""
    
    # Setting the input fields
    text = dspy.InputField(desc = 'The text to be analyzed', default = '')
    language = dspy.InputField(desc = 'The language of the text', default = 'English')
    
    # Setting the output fields
    sentiment = dspy.OutputField(desc = 'The sentiment of the text (positive, negative, or neutral)', default = '')
    topic = dspy.OutputField(desc = 'The main topic of the text', default = '')
    formality = dspy.OutputField(desc = 'The formality level (formal, informal, or neutral)', default = '')
    word_count = dspy.OutputField(type = int, desc = 'The number of words in the text', default = 0)

# Creating a module using our custom signature
dspy_text_analyzer = dspy.Predict(TextAnalysisSignature)

# Creating sample texts for analysis
business_email = "Dear Ms. Smith, I wanted to inform you about the upcoming changes to our project timeline. Please review the attached document for details."
casual_message = "Hey there! Just wanted to check in and see how you're doing. Let's catch up soon!"

# Analyzing the texts
business_analysis = dspy_text_analyzer(text = business_email, language = "English")
casual_analysis = dspy_text_analyzer(text = casual_message, language = "English")

# Displaying analysis results in a more creative way
def display_analysis(title, analysis):
    width = 50
    print("=" * width)
    print(f" {title} ".center(width, "*"))
    print("=" * width)
    print(f"📊 SENTIMENT: {analysis.sentiment}")
    print(f"📝 TOPIC: {analysis.topic}")
    print(f"🎩 FORMALITY: {analysis.formality}")
    print(f"🔢 WORD COUNT: {analysis.word_count}")
    print("-" * width)
    
display_analysis("BUSINESS EMAIL ANALYSIS", business_analysis)
print("\n")
display_analysis("CASUAL MESSAGE ANALYSIS", casual_analysis)

************ BUSINESS EMAIL ANALYSIS *************
📊 SENTIMENT: Positive
📝 TOPIC: Project changes and attached document
🎩 FORMALITY: Formal
🔢 WORD COUNT: 10
--------------------------------------------------


************ CASUAL MESSAGE ANALYSIS *************
📊 SENTIMENT: Positive
📝 TOPIC: Checking in and catching up
🎩 FORMALITY: Informal
🔢 WORD COUNT: 6
--------------------------------------------------
